## Ceneo Scraper

## Ekstrakcja składowych pojedyńczej tabeli

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post_author-recomendation|recommendations|
|gwiazdki|span.user-post_score_count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-future__title--positives|pros|
|lista wad|div.review-future__title--negatives|cons|
|dla ilu przydatna|span[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|span[id^="votes-no"]|useless|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|

## Biblioteki

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Funkcja do ekstrakcji danych ze strony HTML

In [2]:
def extract(ancestor, selector, attribute=None, returns_list=False):
    if selector:
        if returns_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]   
            return [tag.get_text().strip() for tag in ancestor.select(selector)]    

        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None

        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

## Słownik reprezentujący strukturę opinii w serwisie Ceneo.pl

In [3]:
selectors = {
    "opinion_id": (None, "data-entry-id"),
    "author": ("span.user-post__author-name",),
    "recommendation": ('span.user-post__author-recomendation > em',),
    "rating":  ('span.user-post__score-count',),
    "content":  ('div.user-post__text',),
    "pros": ('div.review-feature__title--positives ~ div.review-feature__item', None, True),
    "cons": ('div.review-feature__title--negatives ~ div.review-feature__item', None, True),
    "useful":  ('span[id^="votes-yes"]',),
    "useless":  ('span[id^="votes-no"]',),
    "post_date": ('span.user-post__published > time:nth-child(1)',"datetime"),
    "purchase_date": ('span.user-post__published > time:nth-child(2)',"datetime")
}

## Link do pierwszej strony z opiniami o wskazanym produkcie

In [4]:
# product_id = "114700014"
product_id = input("Proszę podać kod produktu z serwisu Ceneo.pl")
url = f"https://www.ceneo.pl/{product_id}/opinie-1"
response = requests.get(url)

## Pobranie wszystkich opinii o wskazanym produkcie

In [5]:
all_opinions = []
while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None

## Zapis opinii do pliku JSON

In [6]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [7]:
len(all_opinions)

185